# Solenoid Modeling

We can create solenoid fieldmesh data using an ideal model described in:

Derby, N., & Olbert, S. (2010). Cylindrical magnets and ideal solenoids.
American Journal of Physics, 78(3), 229–235. https://doi.org/10.1119/1.3256157

(preprint: https://arxiv.org/abs/0909.3880)


In [ ]:
from pmd_beamphysics.fields.solenoid import make_solenoid_fieldmesh, fit_ideal_solenoid

from pmd_beamphysics.fields.analysis import solenoid_analysis

from pmd_beamphysics.fields.analysis import check_static_div_equation
from pmd_beamphysics.units import mu_0

from pmd_beamphysics import FieldMesh

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
FM = make_solenoid_fieldmesh(
    radius=0.05,
    L=0.2,
    rmax=0.1,
    zmin=-0.5,
    zmax=0.5,
    nr=51,
    nz=100,
    B0=1,
)

In [ ]:
FM.plot()

In [ ]:
FM.plot(stream=True, mirror="r")

In [ ]:
FM.plot_onaxis()

This calculates field integrals, and gives the equivalent hard-edge solenoid parameters:

In [ ]:
z0, Bz0 = FM.axis_values("z", "Bz")
solenoid_analysis(z0, Bz0)

Check the static divergence Maxwell equation

In [ ]:
check_static_div_equation(FM, rtol=1e-1, plot=True)

## Hard edge

Making the radius very small approximates a hard-edge model.

Here we expext that $B_z = \mu_0 n I$

In [ ]:
FM_hard = make_solenoid_fieldmesh(
    radius=1e-9,
    L=0.2,
    rmax=0.1,
    zmin=-0.4,
    zmax=0.4,
    nr=101,
    nz=200,
    nI=1,
)

In [ ]:
FM_hard.Bz[0, 0, :].max() == mu_0

In [ ]:
FM_hard.plot_onaxis()

Check hard-edge analysis again

In [ ]:
z0, Bz0 = FM_hard.axis_values("z", "Bz")
solenoid_analysis(z0, Bz0)

## Compare with a real solenoid

In [ ]:
FM2 = FieldMesh("../data/solenoid.h5")
FM2.plot_onaxis()

In [ ]:
z0, Bz0 = FM2.axis_values("z", "Bz")

fit = fit_ideal_solenoid(z0, Bz0)
fit

In [ ]:
FM3 = make_solenoid_fieldmesh(
    radius=fit["radius"],
    L=fit["L"],
    rmax=0.1,
    zmin=-0.1,
    zmax=0.1,
    nr=100,
    nz=40,
    B0=fit["B0"],
)

In [ ]:
fig, ax = plt.subplots()
z0, Bz0 = FM2.axis_values("z", "Bz")
Bz0 = np.real(Bz0 / Bz0.max())
ax.plot(z0, Bz0, label="Superfish")

z, Bz = FM3.axis_values("z", "Bz")
Bz = np.real(Bz / Bz.max())
ax.plot(z, Bz, label="Ideal")
plt.legend()
ax.set_ylim(0, None)
ax.set_xlabel(r"$z$ (m)")
ax.set_ylabel(r"$B_z$ (T)")

Note that the fields are very different off-axis:

In [ ]:
FM2.plot()
FM3.plot()

# Stylized Plot

Here we make a stylized plot, similar to the figure Fig. 1 in Derby & Olbert (2010).

In [ ]:
FM = make_solenoid_fieldmesh(
    radius=0.05,
    L=0.2,
    rmax=0.5,
    zmin=-0.5,
    zmax=0.5,
    nr=200,
    nz=200,
    B0=1,
)

In [ ]:
# Stylize with varying line thickness
linewidth = 0.5 + np.random.normal(scale=0.3, size=(2 * FM.shape[0] - 1, FM.shape[2]))

FM.plot(
    stream=True,
    mirror="r",
    density=8,
    linewidth=linewidth,
    arrowsize=0,
    figsize=(8, 8),
    aspect="equal",
    cmap="copper",
)

In [ ]:
"acfa_".removesuffix("_")